# Combinación de cadenas LCEL

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_core.messages.human import HumanMessage
from langchain_core.messages.ai import AIMessage
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel

_ = load_dotenv(find_dotenv())
openai_api_key = os.environ['OPENAI_API_KEY']
model = ChatOpenAI(model="gpt-4o-mini")
output_parser = StrOutputParser()

## Coerción: una cadena dentro de otra cadena
* Recuerde: casi cualquier componente de LangChain (prompts, modelos, output parsers, etc.) se puede utilizar como un ejecutable.
* **Los Runnables se pueden encadenar entre sí mediante el operador de barra vertical `|`. Las cadenas de ejecutables resultantes también son ejecutables en sí mismas**.

In [2]:
prompt = ChatPromptTemplate.from_template("Dime una frase sobre {politician}")


chain = prompt | model | StrOutputParser()

In [3]:
chain.invoke("Percy Tejada")

'Percy Tejada es un destacado personaje que ha dejado huella en su comunidad a través de su compromiso y dedicación.'

#### Coerción: combina una cadena con otros Runnables para crear una nueva cadena.
* Observa cómo en la `composed_chain` estamos incluyendo la `chain` anterior:

In [4]:
historian_prompt = ChatPromptTemplate.from_template("Fue el {politician} positivo para la Humanidad?")
composed_chain = {"politician": chain} | historian_prompt | model | StrOutputParser()

In [5]:
composed_chain.invoke({"politician": "Lincoln"})

'Sí, el liderazgo de Abraham Lincoln es generalmente considerado como un aspecto positivo para la humanidad. Su papel en la abolición de la esclavitud y en la promoción de la igualdad contribuyó significativamente a avanzar en los derechos humanos y en la justicia social. La Emancipación de los esclavos y su firme postura contra la división del país durante la Guerra Civil estadounidense ayudaron a sentar las bases para un futuro en el que los derechos de las personas, independientemente de su raza, fueran reconocidos y protegidos.\n\nLincoln defendió ideales de libertad y unidad, y su legado ha influido en movimientos por los derechos civiles y la igualdad en todo el mundo. Sin embargo, también es importante reconocer que su presidencia fue compleja y que sus decisiones a veces generaron críticas. Aun así, en términos generales, su impacto se considera positivo en la lucha por la justicia y la igualdad.'

## Otro ejemplo: una cadena dentro de otra cadena

In [6]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt1 = ChatPromptTemplate.from_template("¿De qué país es {politician}?")
prompt2 = ChatPromptTemplate.from_template(
    "¿De qué continente es el país {country}? responde en {language}"
)

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"country": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

chain2.invoke({"politician": "Miterrand", "language": "French"})

'François Mitterrand était un homme politique français, donc il est de France, qui est un pays situé en Europe.'

## Fallback para cadenas
* Al trabajar con modelos de lenguaje, es posible que a menudo encuentre problemas con las API subyacentes, ya sea por limitación de velocidad o tiempo de inactividad. Por lo tanto, a medida que avanza para trasladar sus aplicaciones LLM a producción, se vuelve cada vez más importante protegerse contra estos problemas. Es por eso que LangChain introdujo el concepto de fallbacks.
* Un fallback es un plan alternativo que se puede utilizar en caso de emergencia.
* Los fallbacks se pueden aplicar no solo en el nivel LLM sino en todo el nivel ejecutable. Esto es importante porque muchas veces los diferentes modelos requieren diferentes indicaciones. Por lo tanto, si su llamada a OpenAI falla, no solo querrá enviar la misma indicación a Anthropic, probablemente desee usar una plantilla de indicación diferente y enviar una versión diferente allí.
* Podemos crear fallbacks para cadenas LCEL. Aquí lo hacemos con dos modelos diferentes: ChatOpenAI (con un nombre de modelo incorrecto para crear fácilmente una cadena que genere un error) y luego OpenAI normal (que no usa un modelo de chat). Dado que OpenAI NO es un modelo de chat, es probable que desees un mensaje diferente.

In [7]:
# Primero, creemos una cadena con un ChatModel
# Agregamos aquí un analizador de salida de cadena para que las salidas entre los dos sean del mismo tipo.

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Eres un asistente divertido que siempre incluye un chiste en su respuesta.",
        ),
        ("human", "¿Quién es el mejor jugador mundial en {sport}?"),
    ]
)
# Aquí vamos a utilizar un nombre de modelo incorrecto para crear fácilmente una cadena que generará un error.
chat_model = ChatOpenAI(model="gpt-fake")

bad_chain = chat_prompt | chat_model | StrOutputParser()

In [8]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

prompt_template = """
Instrucciones: Eres un asistente divertido que siempre incluye un chiste en su respuesta.

Pregunta: ¿Quién es el mejor jugador en {sport}?
"""

prompt = PromptTemplate.from_template(prompt_template)

llm = OpenAI()

good_chain = prompt | llm

In [9]:
# Ahora podemos crear una cadena final que combine los dos
chain = bad_chain.with_fallbacks([good_chain])

chain.invoke({"sport": "soccer"})

'\nRespuesta: ¡El árbitro, porque siempre marca goles!'